In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
import analyze_utils

# save_dir =  '/home/jxm3/research/prompting/interpretable-autoprompting/results_icml/classification__prefix_after_input_2/'
# r, all_losses = analyze_utils.load_results_and_cache_autoprompt_json(
#     save_dir, include_losses=True, do_reranking=False, save_file='r.pkl')

import pickle
r = pickle.load(open('../results/classification/r.pkl', 'rb'))

In [3]:
len(r)

30867

In [4]:
pd.set_option('display.max_colwidth', None)

top_prompts = (
    r [r['num_learned_tokens'] < 24] # Drop 24-token runs because some autoprompt runs failed (OOM).
      # .sort_values(by='prefix_train_acc', ascending=False)
      .sort_values(by='prefix_train_loss', ascending=True)
      .groupby(['num_learned_tokens', 'task_name', 'model_cls', 'seed'])
    
).first()
print(len(top_prompts))


top_prompts[['prefixes', 'reciprocal_rank', 'prefix_train_loss', 'prefix_train_acc']]

48


prefixes  \
num_learned_tokens task_name  model_cls  seed                                                                              
6                  ffb_train  autoprompt 1                                                EFFverbal EUR Thorntonshopnown   
                                         2                                             Fur resultolandgroundur augmented   
                                         3                                                      Hackmmmmajoreryitprofits   
                              iprompt    1                                                    almost neutral. However, "   
                                         2                                                              "So, a bottle of   
                                         3                                                 "Does this represent a market   
                   imdb_train autoprompt 1                                             CRIP deserves PIN SOC sling level   
                                         2                                                    as ​Overall': large points   
                                         3                               ™:Supplement Reasons****************RatingUltra   
                              iprompt    1                                                 When you watch and enjoy this   
                                         2                                                      I find this film a total   
                                         3                                                   To summarize this review! :   
                   rt_train   autoprompt 1                                         Whether{{ anotherath<|endoftext|> how   
                                         2                                    congratulations Named #SPONSOREDReport the   
                                         3                                                     wow some oneendered  very   
                              iprompt    1                                                  "not only are the characters   
                                         2                                                    Who is the author of these   
                                         3                                                   Do you agree with the above   
                   sst2_train autoprompt 1                                         \t BryceSpecificallyWASHINGTONRatedam   
                                         2                                                396 trulyCustomer echoes the "   
                                         3                                            ".Too organic appeal "… thoroughly   
                              iprompt    1                                              Can be used to describe anything   
                                         2                                         A statement that expresses a definite   
                                         3                                               Use this sentence to express an   
12                 ffb_train  autoprompt 1                    proportstals"]," AoErisome peas(" Argentina balance WININc   
                                         2                 oil feed UsingOilalyst Albert Herb Grass ling Bankingthe mild   
                                         3          izationalquartersLord quarterTableHeadperiodMON goTEXT Sylcommercial   
                              iprompt    1                                     <input> neutral> The result was due to: "   
                                         2                                A neutral sentence. Should it be: "This is the   
                                         3                                     Neutral? Hmmm. Let's think about this. It   
                   imdb_train autoprompt 1                    Luaagram RomanFaith Rockyux meets Cast Writing Rating and=   
                    

In [5]:
top_prompts.groupby(['task_name', 'model_cls']).mean()['prefix_train_acc']

task_name   model_cls 
ffb_train   autoprompt    0.692708
            iprompt       0.804688
imdb_train  autoprompt    0.888021
            iprompt       0.908854
rt_train    autoprompt    0.781250
            iprompt       0.828125
sst2_train  autoprompt    0.835938
            iprompt       0.869792
Name: prefix_train_acc, dtype: float64

In [20]:
# assert r['checkpoint'].unique()[0] == "EleutherAI/gpt-j-6B"

# from iprompt import prompt_classification

# model = prompt_classification.create_model(r['checkpoint'].unique()[0])

In [ ]:
## Compute accuracy given correct prompt and save for each task.
import argparse
from tqdm.notebook import tqdm
from iprompt.data import get_data


data = []
print('calculating accs...')
n_shots = 1
batch_size = 8

"""
task_name: str = 'add_two',
 n_shots: int = 1,
 train_split_frac: float = None,
 max_dset_size: int = 10000,
 template_num_task_phrasing: int = 0,
 max_digit: int = 10,
 """

for _, output in tqdm(top_prompts.reset_index().iterrows(), total=len(top_prompts)):
    verbose = False
    max_length = 128
    # if not (('ffb' in output['task_name']) or ('imdb' in output['task_name'])): continue
    # if output['model_cls'] == 'autoprompt': continue
    # if 'ffb' in output['task_name']: continue
    if output['num_learned_tokens'] <= 6: continue # skip that eval for now :-)
    output['task_name'] = output['task_name'].replace('_train', '_test')
    args = argparse.Namespace(**output)
    args.train_split_frac = 1.0 # take 100% of test set
    args.max_dset_size = 1_000
    print("*-*-" * 20)
    print(args.task_name)
    (dset, __dset_test), check_answer_func, descr = get_data(
        args.task_name, n_shots=n_shots, train_split_frac=args.train_split_frac,
        max_dset_size=args.max_dset_size, template_num_task_phrasing=0,
    )
    # if task_name == 'task107_splash_question_to_sql':
    #     batch_size = max(1, batch_size//4)
    ####   Manual prompt  ####
    descr = "" # tmp override
    manual_loss, manual_acc = prompt_classification.test_model_on_task_with_prefix(
        dset=dset, model=model, prefix=descr, multi_token=False, verbose=verbose,
        max_length=max_length, batch_size=64, tqdm_notebook=True,
        restrict_to_valid_answers=True,
        prefix_before_input=False,
    )
    # print(output)
    print(f'\t{descr} || {manual_acc:.1f}%')
    ####   iPrompt prompt   ####
    iprompt_loss, iprompt_acc = prompt_classification.test_model_on_task_with_prefix(
        dset=dset, model=model, prefix=output['prefixes'], multi_token=False, verbose=verbose,
        max_length=max_length, batch_size=64, tqdm_notebook=True,
        restrict_to_valid_answers=True,
        prefix_before_input=False,
    )
    print(f'\t{output["prefixes"]} || {iprompt_acc:.1f}%')
    ####
    output['manual_acc'] = manual_acc
    output['iprompt_acc'] = iprompt_acc
    data.append(output)

In [ ]:
df_with_acc = pd.DataFrame(data)
# df_with_acc[['task_name', 'model_cls', 'seed', 'prefixes', 'prefix_train_acc', 'iprompt_acc', 'manual_acc']]
df_with_acc[df_with_acc['num_learned_tokens'] < 24].groupby(['num_learned_tokens', 'task_name', 'model_cls']).mean()[['iprompt_acc', 'manual_acc']]

In [ ]:
final_prompts_df = df_with_acc[df_with_acc['num_learned_tokens'] < 24]
pd.set_option('display.max_rows', None)

final_prompts_df[['num_learned_tokens', 'model_cls', 'task_name', 'prefixes', 'iprompt_acc']]

In [84]:
final_prompts_df.groupby(['model_cls', 'task_name']).mean()[['iprompt_acc', 'manual_acc']]

iprompt_acc  manual_acc
model_cls  task_name                         
autoprompt ffb_test     56.163636   42.707071
           imdb_test    86.666667   58.600000
           rt_test      73.016667   59.200000
           sst2_test    76.701070   60.894495
iprompt    ffb_test     79.250000   47.500000
           imdb_test    87.916667   58.600000
           rt_test      84.833333   59.200000
           sst2_test    86.716361   60.894495

In [86]:
final_prompts_df.groupby(['model_cls', 'task_name']).sem()[['iprompt_acc', 'manual_acc']]

iprompt_acc  manual_acc
model_cls  task_name                         
autoprompt ffb_test      9.053265    2.443157
           imdb_test     1.204067    0.000000
           rt_test       4.829798    0.000000
           sst2_test     3.901040    0.000000
iprompt    ffb_test      2.028587    0.000000
           imdb_test     1.378506    0.000000
           rt_test       0.829726    0.000000
           sst2_test     0.951405    0.000000

In [26]:
## Promptsource GPT-J Accuracy
from iprompt import prompt_classification
from tqdm.notebook import tqdm
from iprompt.data import get_data

PROMPTSOURCE_PROMPTS = [
    ' The movie review in negative/positive sentiment is:',
    ###############################################################
    ' Does the previous input have a positive or negative sentiment?',
    ' Was that review positive or negative?',
    ' The sentiment expressed for the movie is',
    ' What is the sentiment expressed by the reviewer for the movie?',
    ' How does the viewer feel about the movie?',
    ' Is this review positive or negative?',
    ' What is the sentiment expressed in this text?',
    ' What sentiment does the writer express for the movie?',
]

# Financial phrasebank has a neutral class
PROMPTSOURCE_PROMPTS_NEUTRAL = [
    ' Does the previous input have a positive, neutral, or negative sentiment?',
    ' Was that input positive, neutral, or negative?',
    ' The sentiment expressed in the headline is',
    ' What is the sentiment expressed by the reviewer for the movie?',
    ' How does the author of the news headline feel?',
    ' Is this news headline positive, neutral, or negative?',
    ' What is the sentiment expressed in this text?',
    ' What sentiment does the writer express?',
]


prompt_data = []
task_names = r.reset_index()['task_name'].unique().tolist()
for task_name in tqdm(task_names):
    verbose = False
    max_length = 128
    prompts = PROMPTSOURCE_PROMPTS_NEUTRAL if ('ffb' in task_name) else PROMPTSOURCE_PROMPTS
    for _idx, _prompt in enumerate(prompts):
        output = {}
        output['task_name'] = task_name.replace('_train', '_test')
        train_split_frac = 1.0 # take 100% of test set
        max_dset_size = 1_000
        (dset, __dset_test), check_answer_func, descr = get_data(
            output['task_name'], n_shots=1, train_split_frac=1.0,
            max_dset_size=max_dset_size, template_num_task_phrasing=0,
        )
    #     ####   human-written prompt   ####
        loss, acc = prompt_classification.test_model_on_task_with_prefix(
            dset=dset, model=model, prefix=_prompt, multi_token=False, verbose=verbose,
            max_length=max_length, batch_size=64, tqdm_notebook=True,
            restrict_to_valid_answers=True,
            prefix_before_input=False,
        )
        print(f'\t{_prompt} || {acc:.1f}%')
        ####
        output['prompt_idx'] = _idx
        output['acc'] = acc
        prompt_data.append(output)

  0%|          | 0/4 [00:00<?, ?it/s]

**loading data: sst2 // validation


Found cached dataset sst2 (/home/jxm3/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)
Loading cached shuffled indices for dataset at /home/jxm3/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5/cache-48129d1559bb651f.arrow
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5/cache-93efb9f1d3e163f8.arrow
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5/cache-5b1f2782ad5399c1.arrow


  0%|          | 0/14 [00:00<?, ?it/s]

	 The movie review in negative/positive sentiment is: || 66.1%
**loading data: sst2 // validation


...

Found cached dataset financial_phrasebank (/home/jxm3/.cache/huggingface/datasets/financial_phrasebank/sentences_allagree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141)
Loading cached shuffled indices for dataset at /home/jxm3/.cache/huggingface/datasets/financial_phrasebank/sentences_allagree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141/cache-5e700914242e73b3.arrow


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1698 [00:00<?, ?ex/s]

  0%|          | 0/16 [00:00<?, ?it/s]

	 What sentiment does the writer express? || 25.4%


In [31]:
human_prompt_df = pd.DataFrame(prompt_data)
human_prompt_df.groupby('task_name').mean()

,prompt_idx,acc
task_name,,
ffb_test,3.5,26.250000
imdb_test,4.0,65.833333
rt_test,4.0,52.722222
sst2_test,4.0,54.001019


# Testing with GPT-3

In [ ]:
## Compute accuracy given correct prompt and save for each task.
import os
os.environ['OPENAI_API_KEY'] = 'SECRET'

gpt3_model = prompt_classification.create_model('gpt3')

import argparse
from tqdm.notebook import tqdm
from iprompt.data import get_data


# data = []
print('calculating accs...')
n_shots = 1
batch_size = 8

"""
task_name: str = 'add_two',
 n_shots: int = 1,
 train_split_frac: float = None,
 max_dset_size: int = 10000,
 template_num_task_phrasing: int = 0,
 max_digit: int = 10,
 """

for _, output in tqdm(top_prompts.reset_index().iterrows(), total=len(top_prompts)):
    verbose = False
    max_length = 128
    if output['num_learned_tokens'] <= 24: continue # skip that eval
    output['task_name'] = output['task_name'].replace('_train', '_test')
    args = argparse.Namespace(**output)
    args.train_split_frac = 1.0 # take 100% of test set
    args.max_dset_size = 1000
    print("*-*-" * 20)
    print(args.task_name)
    (dset, __dset_test), check_answer_func, descr = get_data(
        args.task_name, n_shots=n_shots, train_split_frac=args.train_split_frac,
        max_dset_size=args.max_dset_size, template_num_task_phrasing=0,
    )
    # if task_name == 'task107_splash_question_to_sql':
    #     batch_size = max(1, batch_size//4)
    ####   Manual prompt  ####
    descr = "" # tmp override
    manual_loss, manual_acc = prompt_classification.test_model_on_task_with_prefix(
        dset=dset, model=gpt3_model, prefix=descr, multi_token=False, verbose=verbose,
        max_length=max_length, batch_size=64, tqdm_notebook=True,
        restrict_to_valid_answers=True,
        prefix_before_input=False,
    )
    print(output)
    print(f'\t{descr} || {manual_acc:.1f}%')
    ####   iPrompt prompt   ####
    iprompt_loss, iprompt_acc = prompt_classification.test_model_on_task_with_prefix(
        dset=dset, model=gpt3_model, prefix=output['prefixes'], multi_token=False, verbose=verbose,
        max_length=max_length, batch_size=64, tqdm_notebook=True,
        restrict_to_valid_answers=True,
        prefix_before_input=False,
    )
    print(f'\t{output["prefixes"]} || {iprompt_acc:.1f}%')
    ####
    output['manual_acc'] = manual_acc
    output['iprompt_acc'] = iprompt_acc
    data.append(output)
    if _ > 4: break

In [46]:
import transformers
tt = transformers.AutoTokenizer.from_pretrained('gpt2')
tt.vocab_size

50257

# Loading with PromptSource

In [98]:
import promptsource
import promptsource.templates

imdb_prompts = promptsource.templates.DatasetTemplates('imdb')
# ffb_prompts = promptsource.templates.DatasetTemplates('financial_phrasebank', 'sentences_allagree')

pos_input = { "text": "\"What a wonderful film :) \"", "label": 1 }
neg_input = { "text": "\"This movie sucks!\"", "label": 0 }

for tn in imdb_prompts.all_template_names:
    print(tn)
    print('\t [+]', imdb_prompts[tn].apply(pos_input))
    print('\t [-]', imdb_prompts[tn].apply(neg_input))
    print()

Movie Expressed Sentiment
	 [+] ['"What a wonderful film :) " The sentiment expressed for the movie is', 'positive']
	 [-] ['"This movie sucks!" The sentiment expressed for the movie is', 'negative']

Movie Expressed Sentiment 2
	 [+] ['The following movie review expresses what sentiment? "What a wonderful film :) "', 'positive']
	 [-] ['The following movie review expresses what sentiment? "This movie sucks!"', 'negative']

Negation template for positive and negative
	 [+] ['"What a wonderful film :) " This is definitely not a', 'negative review.']
	 [-] ['"This movie sucks!" This is definitely not a', 'positive review.']

Reviewer Enjoyment
	 [+] ['"What a wonderful film :) " How does the reviewer feel about the movie?', 'They loved it']
	 [-] ['"This movie sucks!" How does the reviewer feel about the movie?', "They didn't like it!"]

Reviewer Enjoyment Yes No
	 [+] ['"What a wonderful film :) " Did the reviewer enjoy the movie?', 'Yes']
	 [-] ['"This movie sucks!" Did the reviewer en

In [20]:
imdb_prompts["Movie Expressed Sentiment"].apply({ "text": "This movie sucks!" })

['This movie sucks! The sentiment expressed for the movie is', '']